In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
import pickle

from model_creation import create_smote_data, fit_val_data, model_data, perform_base_models
from model_creation import perform_feat_sel_models, perform_grid_search

In [3]:
train_data = pd.read_csv('C:\\DePaulCoursework\\Winter 2019 CSC 529\\Project\\Data\\CervicalCancerRisks_Combo_target_TrainData.csv', 
                          delimiter=",", index_col = 0, header = 0)
train_target = pd.read_csv('C:\\DePaulCoursework\\Winter 2019 CSC 529\\Project\\Data\\CervicalCancerRisks_Combo_target_TrainTarget.csv', 
                          delimiter=",", index_col = 0, header = 0)

In [4]:
val_data = pd.read_csv('C:\\DePaulCoursework\\Winter 2019 CSC 529\\Project\\Data\\CervicalCancerRisks_Combo_target_ValData.csv', 
                          delimiter=",", index_col = 0, header = 0)
val_target = pd.read_csv('C:\\DePaulCoursework\\Winter 2019 CSC 529\\Project\\Data\\CervicalCancerRisks_Combo_target_ValTarget.csv', 
                          delimiter=",", index_col = 0, header = 0)

In [5]:
rand_st = 1


In [6]:
train_data_sm1p0, train_target_sm1p0 = create_smote_data(train_data, train_target, rand_st, 1.0)

Number of Original Target 0 Value: 506
Number of Original Target 1 Value: 74


Number of SMOTE Target 0 Value: 506
Number of SMOTE Target 1 Value: 506


In [7]:
#train_sm_data_name = data_save_path + 'CervicalCancerRisks_' + target_name + '_TrainData_SMOTE.csv'
#data_train_sm1p0.to_csv(train_sm_data_name, sep = ',')
#train_sm_target_name = data_save_path + 'CervicalCancerRisks_' + target_name + '_TrainTarget_SMOTE.csv'
#target_train_sm1p0.to_csv(train_sm_target_name, sep = ',')


In [8]:
scorers = {'Accuracy': 'accuracy', 'roc_auc': 'roc_auc'}
thresh_vals = [0.4, 0.5, 0.6]
k_vals = [5, 6, 7, 8, 9, 10]

In [9]:
model_type = 'Gradient Boosting'
clf_base = GradientBoostingClassifier(n_estimators = 100, loss = 'deviance', 
                                 learning_rate = 0.1, max_depth = 3,  
                                 min_samples_split = 3, random_state = rand_st)


In [10]:
perform_base_models(train_data, train_target, val_data, val_target, train_data_sm1p0, train_target_sm1p0, clf_base, 
                    scorers, model_type)

Base Gradient Boosting CV Acc: 0.8603407270723613 +/- 0.012987476759374889
Base Gradient Boosting CV AUC: 0.6089819822318365 +/- 0.18527008080501953
CV Runtime: 1.20033597946167


Base Gradient Boosting Validation Acc: 0.8424657534246576
Base Gradient Boosting Validation AUC: 0.48425196850393704


SMOTE Base Gradient Boosting CV Acc: 0.8868569209862163 +/- 0.25018064520691274
SMOTE Base Gradient Boosting CV AUC: 0.9594914262915551 +/- 0.1189282923675929
CV Runtime: 1.9488904476165771


SMOTE Base Gradient Boosting Validation Acc: 0.8493150684931506
SMOTE Base Gradient Boosting Validation AUC: 0.4881889763779528




(GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=3,
               min_weight_fraction_leaf=0.0, n_estimators=100,
               n_iter_no_change=None, presort='auto', random_state=1,
               subsample=1.0, tol=0.0001, validation_fraction=0.1,
               verbose=0, warm_start=False),
 GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=3,
               min_weight_fraction_leaf=0.0, n_estimators=100,
               n_iter_no_change=None, presort='auto', random_state=1,
        

In [11]:
results_dict_base = \
  perform_feat_sel_models(train_data, train_target, val_data, val_target, clf_base, model_type, thresh_vals, k_vals)

-- Low Variance Filtering --
Threshold Selected: 0.4
Selected Model Mean Accuracy Score: 0.8551533207755095
Selected Model Accuracy Deviation: 0.0232834764378063
Selected Model Mean AUC Score: 0.5492153164896322
Selected Model AUC Deviation: 0.13094512131290964
Number of Original Features: 30
Number of Selected Features: 8
Features Selected:
['age', 'num_sex_partners', 'first_sex_int', 'num_pregnancies', 'smokes_yrs', 'smokes_pk_yrs', 'hormonal_contr_yrs', 'iud_yrs']
Features Removed:
['smokes', 'hormonal_contr', 'iud', 'stds', 'stds_num', 'stds_condylomatosis', 'stds_cerv_condylomatosis', 'stds_vag_condylomatosis', 'stds_vp_condylomatosis', 'stds_syphillus', 'stds_pelvic_inf_disease', 'stds_gen_herpes', 'stds_molluscom_contagiosum', 'stds_aids', 'stds_hiv', 'stds_hep_b', 'stds_hpv', 'stds_num_dx', 'dx_cancer', 'dx_cin', 'dx_hpv', 'dx']


Low Variance Filter Gradient Boosting Validation Acc: 0.8561643835616438
Low Variance Filter Gradient Boosting Validation AUC: 0.4921259842519685


-

In [12]:
model_type = 'SMOTE Gradient Boosting'
results_dict_1p0 = \
  perform_feat_sel_models(train_data_sm1p0, train_target_sm1p0, val_data, val_target, clf_base, model_type, 
                          thresh_vals, k_vals)

-- Low Variance Filtering --
Threshold Selected: 0.4
Selected Model Mean Accuracy Score: 0.8818773053775967
Selected Model Accuracy Deviation: 0.23054455966838283
Selected Model Mean AUC Score: 0.9550262955652515
Selected Model AUC Deviation: 0.13320798929425354
Number of Original Features: 30
Number of Selected Features: 8
Features Selected:
['age', 'num_sex_partners', 'first_sex_int', 'num_pregnancies', 'smokes_yrs', 'smokes_pk_yrs', 'hormonal_contr_yrs', 'iud_yrs']
Features Removed:
['smokes', 'hormonal_contr', 'iud', 'stds', 'stds_num', 'stds_condylomatosis', 'stds_cerv_condylomatosis', 'stds_vag_condylomatosis', 'stds_vp_condylomatosis', 'stds_syphillus', 'stds_pelvic_inf_disease', 'stds_gen_herpes', 'stds_molluscom_contagiosum', 'stds_aids', 'stds_hiv', 'stds_hep_b', 'stds_hpv', 'stds_num_dx', 'dx_cancer', 'dx_cin', 'dx_hpv', 'dx']


Low Variance Filter SMOTE Gradient Boosting Validation Acc: 0.8082191780821918
Low Variance Filter SMOTE Gradient Boosting Validation AUC: 0.4645669

In [13]:
train_data_sm0p3, train_target_sm0p3 = create_smote_data(train_data, train_target, rand_st, 0.3)

Number of Original Target 0 Value: 506
Number of Original Target 1 Value: 74


Number of SMOTE Target 0 Value: 506
Number of SMOTE Target 1 Value: 151


In [14]:
perform_base_models(train_data, train_target, val_data, val_target, train_data_sm0p3, train_target_sm0p3, clf_base, 
                    scorers, model_type)

Base SMOTE Gradient Boosting CV Acc: 0.8603407270723613 +/- 0.012987476759374889
Base SMOTE Gradient Boosting CV AUC: 0.6089819822318365 +/- 0.18527008080501953
CV Runtime: 1.2672722339630127


Base SMOTE Gradient Boosting Validation Acc: 0.8424657534246576
Base SMOTE Gradient Boosting Validation AUC: 0.48425196850393704


SMOTE Base SMOTE Gradient Boosting CV Acc: 0.844986512081731 +/- 0.12945469156197198
SMOTE Base SMOTE Gradient Boosting CV AUC: 0.8134035984243585 +/- 0.2365738032369845
CV Runtime: 1.463158369064331


SMOTE Base SMOTE Gradient Boosting Validation Acc: 0.8561643835616438
SMOTE Base SMOTE Gradient Boosting Validation AUC: 0.4921259842519685




(GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=3,
               min_weight_fraction_leaf=0.0, n_estimators=100,
               n_iter_no_change=None, presort='auto', random_state=1,
               subsample=1.0, tol=0.0001, validation_fraction=0.1,
               verbose=0, warm_start=False),
 GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=3,
               min_weight_fraction_leaf=0.0, n_estimators=100,
               n_iter_no_change=None, presort='auto', random_state=1,
        

In [15]:
results_dict_0p3 = \
  perform_feat_sel_models(train_data_sm0p3, train_target_sm0p3, val_data, val_target, clf_base, model_type, 
                          thresh_vals, k_vals)

-- Low Variance Filtering --
Threshold Selected: 0.4
Selected Model Mean Accuracy Score: 0.8159788784939448
Selected Model Accuracy Deviation: 0.07373713852473832
Selected Model Mean AUC Score: 0.7735265310212236
Selected Model AUC Deviation: 0.24516326644004
Number of Original Features: 30
Number of Selected Features: 8
Features Selected:
['age', 'num_sex_partners', 'first_sex_int', 'num_pregnancies', 'smokes_yrs', 'smokes_pk_yrs', 'hormonal_contr_yrs', 'iud_yrs']
Features Removed:
['smokes', 'hormonal_contr', 'iud', 'stds', 'stds_num', 'stds_condylomatosis', 'stds_cerv_condylomatosis', 'stds_vag_condylomatosis', 'stds_vp_condylomatosis', 'stds_syphillus', 'stds_pelvic_inf_disease', 'stds_gen_herpes', 'stds_molluscom_contagiosum', 'stds_aids', 'stds_hiv', 'stds_hep_b', 'stds_hpv', 'stds_num_dx', 'dx_cancer', 'dx_cin', 'dx_hpv', 'dx']


Low Variance Filter SMOTE Gradient Boosting Validation Acc: 0.8561643835616438
Low Variance Filter SMOTE Gradient Boosting Validation AUC: 0.4921259842

In [18]:
cv = 5
clf_grid = GradientBoostingClassifier(random_state = rand_st)

n_estimators = [50, 100, 250, 500]
loss = ['deviance', 'exponential']
max_depth = [None, 3, 5, 10, 20]
min_samples_split = [3, 5, 7, 10]
learning_rate = [0.01, 0.05, 0.1, 0.3]
max_features  = [None, 'sqrt', 'log2']

#n_estimators = [100]
#loss = ['deviance']
#max_depth = [None]
#min_samples_split = [3]
#learning_rate = [0.1]
#max_features = [None]

model_params = {
    'loss': loss,
    'max_depth': max_depth,
    'n_estimators': n_estimators,
    'min_samples_split': min_samples_split,
    'learning_rate': learning_rate,
    'max_features': max_features
}

score = 'roc_auc'

In [19]:
mutinf_train_data_base = results_dict_base['mutinf']['data']
mutinf_del_cols_base = results_dict_base['mutinf']['del_cols']
mutinf_val_data_base = val_data.drop(mutinf_del_cols_base, axis = 1)

model_type = 'Base Gradient Boost'
model_desc = 'Mutual Information FS'

opt_model = perform_grid_search(clf_grid, mutinf_train_data_base, train_target, model_params, score, cv, 
                              model_type, model_desc)
mutinf_model_sm_base = model_data(mutinf_train_data_base, train_target, mutinf_val_data_base, val_target, 
                                opt_model, scorers, model_type, model_desc, cv)


Mutual Information FS Base Gradient Boost Grid Search
Fitting 5 folds for each of 1920 candidates, totalling 9600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 9600 out of 9600 | elapsed: 221.8min finished
C:\Users\Kari\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Grid Search Runtime: 13307.47797369957


Grid Search Optimal Parameters: {'learning_rate': 0.01, 'loss': 'deviance', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50}
Grid Search Optimal Parameter Score: 0.6689335109257086


Final Model Parameter Settings:
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=20,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=3,
              min_weight_fraction_leaf=0.0, n_estimators=50,
              n_iter_no_change=None, presort='auto', random_state=1,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)


Mutual Information FS Base Gradient Boost CV Acc: 0.8724249413754661 +/- 0.005898386228916201
Mutual Information FS Base Gradient Boost CV AUC: 0.668772129313771

In [20]:
filename = 'Combo_MutInf_FS_Base_GB_Model.sav'
pickle.dump(mutinf_model_sm_base, open(filename, 'wb'))

In [21]:
chisq_train_data_1p0 = results_dict_1p0['chisq']['data']
chisq_del_cols_1p0 = results_dict_1p0['chisq']['del_cols']
chisq_val_data_1p0 = val_data.drop(chisq_del_cols_1p0, axis = 1)

model_type = 'SMOTE 1.0 Gradient Boost'
model_desc = 'Chi-Squared FS'

opt_model = perform_grid_search(clf_grid, chisq_train_data_1p0, train_target_sm1p0, model_params, score, cv, 
                              model_type, model_desc)
chisq_model_sm_1p0 = model_data(chisq_train_data_1p0, train_target_sm1p0, chisq_val_data_1p0, val_target, 
                                opt_model, scorers, model_type, model_desc, cv)


Chi-Squared FS SMOTE 1.0 Gradient Boost Grid Search
Fitting 5 folds for each of 1920 candidates, totalling 9600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 9600 out of 9600 | elapsed: 238.3min finished
C:\Users\Kari\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Grid Search Runtime: 14299.232560157776


Grid Search Optimal Parameters: {'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'min_samples_split': 7, 'n_estimators': 250}
Grid Search Optimal Parameter Score: 0.870089694391434


Final Model Parameter Settings:
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=7,
              min_weight_fraction_leaf=0.0, n_estimators=250,
              n_iter_no_change=None, presort='auto', random_state=1,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)


Chi-Squared FS SMOTE 1.0 Gradient Boost CV Acc: 0.8146088138225587 +/- 0.19422954892611427
Chi-Squared FS SMOTE 1.0 Gradient Boost CV AUC: 0.8705081193651587 +/- 0.212

In [22]:
filename = 'Combo_ChiSq_FS_SMOTE_1p0_GB_Model.sav'
pickle.dump(chisq_model_sm_1p0, open(filename, 'wb'))